![](img/Tsampl.png) ![](img/Tsar.png)

![](img/ADC%20timing.png)





In [5]:
import numpy as np

#Sample time SMP
Tsampl = np.array([2.5, 6.5, 12.5, 24.5, 47.5, 92.5, 247.5, 640.5])

channel_conf = {"Ibat": {"channel" : 1, "Tsample" : Tsampl[4], "Rin" : 120},
                "Vbat": {"channel" : 8, "Tsample" : Tsampl[4], "Rin" : 120},
                "Vpanel": {"channel" : 3, "Tsample" : Tsampl[4], "Rin" : 120},
                "Ipanel": {"channel" : 5, "Tsample" : Tsampl[4], "Rin" : 120},
                "TempDiode": {"channel" : 9, "Tsample" : Tsampl[3], "Rin" : 1e3},
                "TempMos": {"channel" : 10, "Tsample" : Tsampl[3], "Rin" : 1e3},
                "Rad": {"channel" : 11, "Tsample" : Tsampl[3], "Rin" : 1e3},
                }

# ADC channel sampling order
rank = ["Ibat", "Vbat", "Vpanel", "Ipanel", "TempDiode", "TempMos", "Rad"]

ADC_Res = "12" # ADC Resolution
Tsar = {"12" : 12.5, "10" : 10.5, "8" : 8.5, "6" : 6.5}     # ADC Resolution time SAR

Tadc_clk = 1/80e6 # ADC clock time 1/ADC frequency

Trigger = 7e3  # Trigger frequency

Cadc = 5e-12    #ADC internal capacitor

oversampling_ratio = 32
max_value = oversampling_ratio * ((2**12) -1)
min_bit_shift = (16, max_value.bit_length())[max_value.bit_length() > 16] - 16
max_value_shifted = max_value >> min_bit_shift

Tconv = []
error = []
for channel in rank:
    Tconv.append(oversampling_ratio * (Tsar[ADC_Res] + channel_conf[channel]["Tsample"]) * Tadc_clk)
    # Vo = 1 - exp(-t/RC)
    error.append((np.exp(-(Tadc_clk * channel_conf[channel]["Tsample"]) / (channel_conf[channel]["Rin"] * Cadc))))


print("Tbussy: ", sum(Tconv))
print("Tfree: ", (1/Trigger) - sum(Tconv))
print("Tfree(%)", 100*((1/Trigger) - sum(Tconv)) / (1/Trigger) )
print("Bit gain", np.log(oversampling_ratio) / np.log(4))
print("maximum value (oversample): ", hex(max_value))
print("maximum value shifted (oversample)", hex(max_value_shifted))

print("========== CONF ==========")
print("Resolution: ", ADC_Res)
print("Oversampling: ", oversampling_ratio, "x")
print("Minimum Bit Shift", min_bit_shift)
print("Number of conversion: ", len(rank))
for channel in rank:
    print("Channel: ",channel)
    print("Channel °N: ", channel_conf[channel]["channel"])
    print("Sampling Time: ", channel_conf[channel]["Tsample"])






Tbussy:  0.0001404
Tfree:  2.4571428571428704e-06
Tfree(%) 1.720000000000009
Bit gain 2.5
maximum value (oversample):  0x1ffe0
maximum value shifted (oversample) 0xfff0
========== CONF ==========
Resolution:  12
Oversampling:  32 x
Minimum Bit Shift 1
Number of conversion:  7
Channel:  Ibat
Channel °N:  1
Sampling Time:  47.5
Channel:  Vbat
Channel °N:  8
Sampling Time:  47.5
Channel:  Vpanel
Channel °N:  3
Sampling Time:  47.5
Channel:  Ipanel
Channel °N:  5
Sampling Time:  47.5
Channel:  TempDiode
Channel °N:  9
Sampling Time:  24.5
Channel:  TempMos
Channel °N:  10
Sampling Time:  24.5
Channel:  Rad
Channel °N:  11
Sampling Time:  24.5


In [6]:
%pip install scipy pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import scipy as sci
import pandas as pd

def f(x, a, b):
    return (a * x) + b


df = pd.read_csv('adc.csv')

df.Iin = -df.Iin
popt, pcov = sci.optimize.curve_fit(f, df.Adc, df.Iin)

print(popt)

display(df)
display(f((2**12 * 15), popt[0] * 15, popt[1]))
display(f((0), popt[0], popt[1]))

[ 7.58257820e-02 -1.57422149e+02]


,Iin,Vout,Adc,Vcc,I_calc,Erro,Unnamed: 6,RMSE
0,-0.00,1.672,2075,3.3,-0.3352,23.65384615,#DIV/0!,NaN
1,-1.04,1.660,2060,Adc_resolution,0.7940,39.58503401,23.65%,0.060516
2,-1.47,1.659,2059,4096,0.8881,17.38095238,39.59%,0.338608
3,-2.10,1.650,2048,NaN,1.7350,15.58359621,17.38%,0.133225
4,-3.17,1.640,2036,NaN,2.6760,9.166666667,15.58%,0.244036
5,-4.50,1.625,2017,NaN,4.0875,5.9936,9.17%,0.170156
6,-6.25,1.606,1993,NaN,5.8754,5.426436782,5.99%,0.140325
7,-8.70,1.581,1962,NaN,8.2279,3.343959732,5.43%,0.222878
8,-11.92,1.546,1919,NaN,11.5214,2.488666667,3.34%,0.158882
9,-15.00,1.513,1878,NaN,14.6267,1.698941799,2.49%,0.139353


69723.6185081034

-157.4221487323353

In [8]:
import scipy as sci
import pandas as pd

def f(x, a, b):
    return (a * x) + b


df = pd.read_csv('adc.csv')

df.Iin = -df.Iin
popt, pcov = sci.optimize.curve_fit(f, df.Vout, df.Iin)

print(popt)

display(df)
display(f((2.55), popt[0], popt[1]))

[  94.11291677 -157.42388104]


,Iin,Vout,Adc,Vcc,I_calc,Erro,Unnamed: 6,RMSE
0,-0.00,1.672,2075,3.3,-0.3352,23.65384615,#DIV/0!,NaN
1,-1.04,1.660,2060,Adc_resolution,0.7940,39.58503401,23.65%,0.060516
2,-1.47,1.659,2059,4096,0.8881,17.38095238,39.59%,0.338608
3,-2.10,1.650,2048,NaN,1.7350,15.58359621,17.38%,0.133225
4,-3.17,1.640,2036,NaN,2.6760,9.166666667,15.58%,0.244036
5,-4.50,1.625,2017,NaN,4.0875,5.9936,9.17%,0.170156
6,-6.25,1.606,1993,NaN,5.8754,5.426436782,5.99%,0.140325
7,-8.70,1.581,1962,NaN,8.2279,3.343959732,5.43%,0.222878
8,-11.92,1.546,1919,NaN,11.5214,2.488666667,3.34%,0.158882
9,-15.00,1.513,1878,NaN,14.6267,1.698941799,2.49%,0.139353


82.56405672708797